-CS24MTECH11023 | CHALASANI VINEETH

In [147]:
import csv
import random
import numpy as np
from scipy.linalg import null_space
np.random.seed(0)

In [148]:
def load_data(filename):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
    z = np.array([float(val) for val in data[0][:-1]])
    c = np.array([float(val) for val in data[1][:-1]])
    b = np.array([float(row[-1]) for row in data[2:]])
    A = np.array([[float(val) for val in row[:-1]] for row in data[2:]])
    m, n = len(b), len(c)
    return A, z, b, c, m, n

In [149]:
def calculate_product_az(_A, _z):
  return np.dot(_A, _z)

def get_tight_mask(_product, _b, epi=1e-8):
  return np.abs(_product - _b) < epi

def find_tight_rows(_A, _z, _b, epi=1e-8):
  product = calculate_product_az(_A, _z)
  tight_mask = get_tight_mask(product, _b, epi)
  A1 = _A[tight_mask]
  A2 = _A[~tight_mask]
  return tight_mask, A1, A2

In [150]:
def get_directions(_A1):
  try:
    A_inv = np.linalg.inv(_A1)
    neg_A_inv = -1 * A_inv
    return neg_A_inv
  except np.linalg.LinAlgError:
    print("Matrix is singular. Cannot compute the inverse.")
    return None

In [151]:
def is_degenerate(tight):
  return tight.shape[0] > tight.shape[1]

In [152]:
def make_non_degenerate(vector_b_original, epi, reduction_factor=0.5):
  epi = epi*reduction_factor
  new_b  = np.array([vector_b_original[i]+epi**(i+1) for i in range(len(vector_b_original))])
  return new_b, epi

In [153]:
def get_modified_LP(_A, _b, _c):
  min_b = min(_b)
  initial_point = create_initial_point(_A.shape, min_b)

  modified_A, modified_b, modified_c = _A, _b, _c

  if initial_point.shape[0] != _c.shape[0]:
    modified_A, modified_b, modified_c = modify_matrices_for_LP(_A, _b, _c, min_b)

  return modified_A, modified_b, np.hstack(modified_c), np.hstack(initial_point)

In [154]:
def create_initial_point(A_shape, min_b):
  initial_point = np.zeros((A_shape[1] + 1, 1))
  initial_point[-1] = min_b
  return initial_point

In [155]:
def modify_matrices_for_LP(_A, _b, _c, min_b):
  rows, cols = _A.shape
  modified_A = np.append(np.append(_A, np.zeros((1, cols)), axis=0), np.ones((rows + 1, 1)), axis=1)
  modified_A[-1][-1] = -1
  modified_b = np.append(_b, [abs(min_b)], axis=0)
  modified_c = np.zeros((cols + 1, 1))
  modified_c[-1] = 1
  return modified_A, modified_b, modified_c

In [156]:
def initialize_feasible_to_vertex(_vector_z, _vector_c):
  track_cost = []
  track_z = []
  track_cost.append(np.dot(_vector_c, _vector_z))
  track_z.append(_vector_z)
  z_old = _vector_z
  iteration = 0
  print_interval = 1
  return track_cost, track_z, z_old, iteration, print_interval

In [157]:
def calculate_step_magnitude(_vector_b, mask, untight_rows, z_old, u):
  alphas = [(_b_i - np.dot(a2_i, z_old)) / np.dot(a2_i, u) for _b_i, a2_i in zip(_vector_b[~mask], untight_rows)]
  alphas = [alpha for alpha in alphas if alpha > 0]
  positive_alphas = []
  epi = 1e-10
  for i in alphas:
    if not(i == np.inf or abs(i) < epi):
      positive_alphas.append(i)
  return positive_alphas

In [158]:
def compute_step_direction(matrix_A, tight_rows):
    if len(tight_rows) == 0:
        u = np.random.rand(matrix_A.shape[1])
    else:
        null_space_matrix = null_space(tight_rows)
        if null_space_matrix.shape[1] == 0:
            print("Null space is empty. Cannot compute step direction.")
            return None
        u = null_space_matrix[:, 0]
    return u

In [159]:
def is_vertex(matrix_A, vector_z, vector_b, dimension_n):
  mask, tight_rows, untight_rows = find_tight_rows(matrix_A, vector_z, vector_b)
  if len(tight_rows) == 0:
      rank = 0
  else:
      rank = np.linalg.matrix_rank(tight_rows)
  return rank == dimension_n, mask, tight_rows, untight_rows, rank

In [160]:
def get_direction(_matrix_A, _vector_z, _vector_b, tight_rows):
  if len(tight_rows) == 0:
    u = np.random.rand(_matrix_A.shape[-1])
  else:
    null_space_matrix = null_space(tight_rows)
    u = null_space_matrix[:, 0]
  return u

In [161]:
def initialize_feasible_to_vertex(_vector_z, _vector_c):
  track_cost = []
  track_z = []
  track_cost.append(np.dot(_vector_c, _vector_z))
  track_z.append(_vector_z)
  z_old = _vector_z
  iteration = 0
  print_interval = 1
  return track_cost, track_z, z_old, iteration, print_interval

def iterate_to_vertex(_matrix_A, _vector_b, _vector_c, _dimension_n, z_old, track_cost, track_z, iteration, print_interval):
  mask, tight_rows, untight_rows = find_tight_rows(_matrix_A, z_old, _vector_b)

  if len(tight_rows) == 0:
      rank = 0
  else:
      rank = np.linalg.matrix_rank(tight_rows)

  while rank != _dimension_n:
    iteration += 1

    if iteration % print_interval == 0:
        print(f"Iteration: {iteration} - Rank: {rank}")
        if iteration > 300:
          print_interval = 1000
        elif iteration > 10000:
          print_interval = 10000

    u = get_direction(_matrix_A, z_old, _vector_b, tight_rows)

    while True:
      positive_alphas = calculate_step_magnitude(_vector_b, mask, untight_rows, z_old, u)

      if len(positive_alphas) == 0:
        u = -1*u
      else:
        break

    alpha = min(positive_alphas)

    z_new = z_old + alpha * u

    mask, tight_rows, untight_rows = find_tight_rows(_matrix_A, z_new, _vector_b)

    z_old = z_new

    if len(tight_rows) == 0:
        rank = 0
    else:
        rank = np.linalg.matrix_rank(tight_rows)

    track_cost.append(np.dot(_vector_c, z_new))
    track_z.append(z_new)

  return z_old, track_cost, track_z, tight_rows

def feasible_to_vertex_assign3(_matrix_A, _vector_b, _vector_z, _vector_c, _dimension_n):
  track_cost, track_z, z_old, iteration, print_interval = initialize_feasible_to_vertex(_vector_z, _vector_c)

  mask, tight_rows, untight_rows = find_tight_rows(_matrix_A, _vector_z, _vector_b)

  if len(tight_rows) == 0:
      rank = 0
  else:
      rank = np.linalg.matrix_rank(tight_rows)

  if rank == _dimension_n:
      return z_old, track_cost, track_z
  else:
    print("Feasible point is not a vertex. Searching for a vertex...")
    z_new, track_cost, track_z, tight_rows = iterate_to_vertex(_matrix_A, _vector_b, _vector_c, _dimension_n, z_old, track_cost, track_z, iteration, print_interval)

  if not(is_degenerate(tight_rows)):
    return (z_new, track_cost, track_z)
  else:
    return (None,)

In [162]:
def initialize_vertex_to_vertex(_vector_z, _vector_c):
  track_cost = []
  track_vertex = []
  z_old = _vector_z
  track_cost.append(np.dot(_vector_c, z_old))
  track_vertex.append(z_old)
  iteration = 0
  print_interval = 1
  return track_cost, track_vertex, z_old, iteration, print_interval

def iterate_vertex_to_vertex(_matrix_A, _vector_b, _vector_c, _columns, z_old, track_cost, track_vertex, iteration, print_interval):
  while True:
    iteration += 1
    if iteration % print_interval == 0:
      print(f"Iteration: {iteration}")

    mask, tight_rows, untight_rows = find_tight_rows(_matrix_A, z_old, _vector_b)

    if is_degenerate(tight_rows):
      return (None,)

    directions = get_directions(tight_rows).T

    positive_directions = []
    for direction in directions:
      if np.dot(direction, _vector_c) > 0:
          positive_directions.append(direction)

    if not positive_directions or iteration > 10:
      return z_old, track_cost, track_vertex

    u = positive_directions[0]

    alphas = [(b_i - np.dot(a2_i, z_old)) / np.dot(a2_i, u) for b_i, a2_i in zip(_vector_b[~mask], untight_rows)]
    alphas = [alpha for alpha in alphas if alpha > 0]

    positive_alphas = []
    epi = 1e-10
    for i in alphas:
      if not(i == np.inf or abs(i) < epi):
        positive_alphas.append(i)


    if len(positive_alphas) == 0 :
      print(f"The problem is unbounded. Can't find a optimal solution!")
      return None, track_cost, track_vertex


    alpha = min(positive_alphas)

    z_new = z_old + alpha * u
    z_old = z_new

    track_cost.append(np.dot(_vector_c, z_new))
    track_vertex.append(z_new)

def vertex_to_vertex_assign3(_matrix_A, _vector_b, _vector_z, _vector_c, _columns):
  track_cost, track_vertex, z_old, iteration, print_interval = initialize_vertex_to_vertex(_vector_z, _vector_c)

  result = iterate_vertex_to_vertex(_matrix_A, _vector_b, _vector_c, _columns, z_old, track_cost, track_vertex, iteration, print_interval)

  return result

In [163]:
def find_initial_feasible_point(A, b, c):
    if np.all(b >= 0):
        print("Using origin as the initial feasible point!\n")
        return (np.zeros(c.shape),)
    else:
        return solve_auxiliary_lp(A, b, c)

def solve_auxiliary_lp(A, b, c):
    aux_A, aux_b, aux_c, initial_z = get_modified_LP(A, b, c)
    num_rows, num_cols = aux_A.shape

    print("Auxiliary LP Details:")
    print("Initial Feasible Point (z):", initial_z)
    print("Cost Vector (c):", aux_c)
    print("Constraint Vector (b):", aux_b)
    print("Matrix A:")
    print(aux_A)
    print(f"Rows: {num_rows} | Columns: {num_cols}\n")

    print("Solving the auxiliary LP!\n")
    return find_optimal_auxiliary_lp_solution(aux_A, aux_b, initial_z, aux_c, num_cols, c.shape[0])

def find_optimal_auxiliary_lp_solution(matrix_A, vector_b_original, initial_z, vector_c, dimension_n, original_dim_n):
    epsilon = 0.1
    attempt = 0
    vector_b = vector_b_original
    while True:
        if attempt > 0:
            print(f"Degeneracy detected. Attempting to handle it. Attempt - {attempt}\n")
            vector_b, epsilon = make_non_degenerate(vector_b_original, epsilon)

        outputs1 = feasible_to_vertex_assign3(matrix_A, vector_b, initial_z, vector_c, dimension_n)
        if len(outputs1) == 1:
            attempt += 1
            continue

        print("Reached a vertex from feasible point in auxiliary LP!\n")
        vertex_z, feas_to_vert_cost_history, feas_to_vert_z_history = outputs1

        print("Searching for optimal vertex in auxiliary LP...\n")
        outputs2 = vertex_to_vertex_assign3(matrix_A, vector_b, vertex_z, vector_c, dimension_n)
        if len(outputs2) == 1:
            attempt += 1
            continue

        optimal_z, vert_to_vert_cost_history, vert_to_vert_z_history = outputs2
        return process_auxiliary_lp_solution(optimal_z, original_dim_n, feas_to_vert_cost_history, feas_to_vert_z_history, vert_to_vert_cost_history, vert_to_vert_z_history)

def process_auxiliary_lp_solution(optimal_z, original_dim_n, feas_to_vert_cost_history, feas_to_vert_z_history, vert_to_vert_cost_history, vert_to_vert_z_history):
    if optimal_z is None:
        print("The auxiliary problem is unbounded!\n")
        print("The original problem is unbounded!\n")
        return (None,)
    else:
        auxiliary_objective_value = np.dot(np.zeros(original_dim_n + 1), optimal_z)
        if abs(auxiliary_objective_value) > 1e-6:
             print("The auxiliary problem optimal value is not zero. The original problem is infeasible!\n")
             return (None,)
        else:
            print("Reached the optimal vertex in auxiliary LP!\n")
            return (optimal_z[:original_dim_n], optimal_z, None, feas_to_vert_cost_history, feas_to_vert_z_history, optimal_z, vert_to_vert_cost_history, vert_to_vert_z_history)

## Reading the input

In [164]:
A, z, b, c, m, n = load_data('testcase_5.csv')

print("Initial Feasible Point (z):", z)
print("Cost Vector (c):", c)
print("Constraint Vector (b):", b)
print("Matrix A:")
print(A)
print(f"Rows: {m} | Columns: {n}")

Initial Feasible Point (z): [5. 2.]
Cost Vector (c): [0. 1.]
Constraint Vector (b): [1. 1.]
Matrix A:
[[ 1.  0.]
 [-1. -1.]]
Rows: 2 | Columns: 2


- OUTPUT

In [165]:
outputs = find_initial_feasible_point(A, b, c)
z = None
z_optimal = None
modified_z_optimal = None
feas2vert_z_all_cost = []
feas2vert_z_all = []
vert2vert_z_all_cost = []
vert2vert_z_all = []

if outputs is not None:
    if len(outputs) > 1: # Case where an optimal solution is found for the auxiliary LP
        z_optimal, modified_z_optimal, z_new, feas2vert_z_all_cost, feas2vert_z_all, z_optimal, vert2vert_z_all_cost, vert2vert_z_all = outputs
        if modified_z_optimal is not None:
            z = z_optimal[:len(c)]
    elif len(outputs) == 1: # Case where find_initial_feasible_point returns (None,) for infeasible/unbounded
        z = None
        modified_z_optimal = None
        
        if len(outputs) == 8:
             _, _, _, feas2vert_z_all_cost, feas2vert_z_all, _, vert2vert_z_all_cost, vert2vert_z_all = outputs

Using origin as the initial feasible point!



In [166]:
# Initialize variables to ensure they're always available for display
feas2vert_z_all = []
feas2vert_z_all_cost = []
vert2vert_z_all = []
vert2vert_z_all_cost = []
z_optimal = None

if modified_z_optimal is None:
    print("Could not find an initial feasible point. The problem may be infeasible or unbounded.")
else:
    matrix_A = A
    vector_b_original = b
    vector_z = z
    vector_c = c
    dimension_n = n

    epsilon = 0.1
    attempt = 0
    vector_b = vector_b_original
    attempt = 0
    while True:
      if attempt > 0:
        print(f"Degeneracy detected. Attempting to handle it. Attempt - {attempt}")
        vector_b, epsilon = make_non_degenerate(vector_b_original, epsilon)

      outputs1 = feasible_to_vertex_assign3(matrix_A, vector_b, vector_z, vector_c, dimension_n)
      if len(outputs1) == 1:
        attempt+=1
        continue

      print("Reached the initial vertex from feasible point!")
      z_new, feas2vert_z_all_cost, feas2vert_z_all = outputs1

      print("Searching for optimal vertex...")
      outputs2 = vertex_to_vertex_assign3(matrix_A, vector_b, z_new, vector_c, dimension_n)
      if len(outputs2) == 1:
        attempt+=1
        continue

      z_optimal, vert2vert_z_all_cost, vert2vert_z_all = outputs2
      if np.all(z_optimal == None):
        print("The problem is unbounded!")
      else:
        print("Reached the optimal vertex!")

      break

Could not find an initial feasible point. The problem may be infeasible or unbounded.


In [167]:
if modified_z_optimal is None:
    print("Could not find an initial feasible point. The problem may be infeasible or unbounded.")
else:
    # Always display the sequences regardless of whether optimal or unbounded
    if len(feas2vert_z_all) > 0:
        print("Feasible point to vertex")
        print("Sequence of vertices visited and objective function values:")
        for i, (vertex, cost) in enumerate(zip(feas2vert_z_all, feas2vert_z_all_cost)):
            print(f"Vertex {i+1}: {vertex}, Objective Value: {cost}")
        print(f"\nInitial vertex: {z}")

    if len(vert2vert_z_all) > 0:
        print("\nVertex to Optimal vertex")
        print("Sequence of vertices visited and objective function values:")
        for i, (vertex, cost) in enumerate(zip(vert2vert_z_all, vert2vert_z_all_cost)):
            print(f"Vertex {i+1}: {vertex}, Objective Value: {cost}")

    # Display final result
    if z_optimal is None:
        print("\nProblem is unbounded")
    else:
        print("\nOptimal vertex:")
        print(z_optimal)

Could not find an initial feasible point. The problem may be infeasible or unbounded.
